<a href="https://colab.research.google.com/github/InowaR/colab/blob/main/ai_audio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
!pip install -q google-cloud-speech
!pip install ipywebrtc

In [54]:
from google.colab import auth

PROJECT_ID = userdata.get('PROJECT_ID')

auth.authenticate_user(project_id=PROJECT_ID)

In [55]:
import google.generativeai as genai
from google.colab import userdata


GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

model = genai.GenerativeModel('gemini-pro')

In [56]:
from google.cloud import speech

def speech_to_text(config: speech.RecognitionConfig, audio: speech.RecognitionAudio) -> speech.RecognizeResponse:
    client = speech.SpeechClient()
    response = client.recognize(config=config, audio=audio)
    return response

def print_response(response: speech.RecognizeResponse):
    for result in response.results:
        print_result(result)

def print_result(result: speech.SpeechRecognitionResult):
    best_alternative = result.alternatives[0]
    print("-" * 80)
    print(f"language_code: {result.language_code}")
    print(f"transcript:    {best_alternative.transcript}")
    print(f"confidence:    {best_alternative.confidence:.0%}")


config = speech.RecognitionConfig(
    language_code="en",
    enable_automatic_punctuation=True,
)

In [58]:
from ipywebrtc import AudioRecorder, CameraStream

from google.colab import output
output.enable_custom_widget_manager()

from IPython.display import Markdown
import textwrap

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [59]:
camera = CameraStream(constraints={'audio': True, 'video':False})
recorder = AudioRecorder(stream=camera)
recorder

AudioRecorder(audio=Audio(value=b'', format='webm'), stream=CameraStream(constraints={'audio': True, 'video': …

In [60]:
recorded_audio = recorder.audio.value
print(recorded_audio)

audio = speech.RecognitionAudio(
    content=recorded_audio,
)

processing_results = speech_to_text(config, audio)
audio_text = processing_results.results[0].alternatives[0].transcript

response = model.generate_content(audio_text)

to_markdown(f'**You**: {audio_text}\n\n**Gemini**:\n{response.text}')

b'\x1aE\xdf\xa3\x9fB\x86\x81\x01B\xf7\x81\x01B\xf2\x81\x04B\xf3\x81\x08B\x82\x84webmB\x87\x81\x04B\x85\x81\x02\x18S\x80g\x01\xff\xff\xff\xff\xff\xff\xff\x15I\xa9f\x99*\xd7\xb1\x83\x0fB@M\x80\x86ChromeWA\x86Chrome\x16T\xaek\xbf\xae\xbd\xd7\x81\x01s\xc5\x87\xe6_{\x9an\r\xe6\x83\x81\x02\x86\x86A_OPUSc\xa2\x93OpusHead\x01\x01\x00\x00\x80\xbb\x00\x00\x00\x00\x00\xe1\x8d\xb5\x84G;\x80\x00\x9f\x81\x01bd\x81 \x1fC\xb6u\x01\xff\xff\xff\xff\xff\xff\xff\xe7\x81\x00\xa3D\x8b\x81\x00\x00\x80\xfb\x83\xfeK\xfe\x0b~\xa7\x0e\xa0N\xf3\xa5\x8aZrs\'\x18\xa7Lu\x13\x12\xf0N\x1aD\xa3\x93\xb1\x88\x9c\x0e\xf5\x84\x88\xc3\t57\xc2E\xc8\x93\xf2\xf1\xf3\xc5f\xd5\x17\x88:EYy\x0e\x02\x0b3Y$\x86\x94"t&6\x19c\xb5\xe5\x81XC\xf5\xa1(\xa4\xf2\xa2x\xae\xe8R\x91\x08-C\xc7"\xd4\x94Q\xefO`\xe3/\xc6W\x0f\x91\xa6\xd7\xfd\x1c\xd3"\xa0\xa2\xb8\xf7\xcf\xb9?\xfe.\x94\xa44\x11\xd3>\xf8\x9d\x04\x1boGA\x16\x9f\x976\xfe\xd0NA\xde\xfd\xaa\xa2\xa2PE@w\x83\x0f_(\xd2\x03\xd5g/{\xc6\x13tMs>\xe98\x9a\xb7\x15\x9d\xaf\xfe\xf2z\xd6\xe7bT\xd1\x

> **You**: Hello.
> 
> **Gemini**:
> Hello there! How can I help you today?

In [62]:
with open("audio.webm", "wb") as webm_file:
   webm_file.write(recorder.audio.value)

with open('audio.webm', 'rb') as webm_file:
    webm_data = webm_file.read()

print(webm_data)

audio = speech.RecognitionAudio(
    content=webm_data,
)

processing_results = speech_to_text(config, audio)
audio_text = processing_results.results[0].alternatives[0].transcript

response = model.generate_content(audio_text)

to_markdown(f'**You**: {audio_text}\n\n**Gemini**:\n{response.text}')

b'\x1aE\xdf\xa3\x9fB\x86\x81\x01B\xf7\x81\x01B\xf2\x81\x04B\xf3\x81\x08B\x82\x84webmB\x87\x81\x04B\x85\x81\x02\x18S\x80g\x01\xff\xff\xff\xff\xff\xff\xff\x15I\xa9f\x99*\xd7\xb1\x83\x0fB@M\x80\x86ChromeWA\x86Chrome\x16T\xaek\xbf\xae\xbd\xd7\x81\x01s\xc5\x87\xe6_{\x9an\r\xe6\x83\x81\x02\x86\x86A_OPUSc\xa2\x93OpusHead\x01\x01\x00\x00\x80\xbb\x00\x00\x00\x00\x00\xe1\x8d\xb5\x84G;\x80\x00\x9f\x81\x01bd\x81 \x1fC\xb6u\x01\xff\xff\xff\xff\xff\xff\xff\xe7\x81\x00\xa3D\x8b\x81\x00\x00\x80\xfb\x83\xfeK\xfe\x0b~\xa7\x0e\xa0N\xf3\xa5\x8aZrs\'\x18\xa7Lu\x13\x12\xf0N\x1aD\xa3\x93\xb1\x88\x9c\x0e\xf5\x84\x88\xc3\t57\xc2E\xc8\x93\xf2\xf1\xf3\xc5f\xd5\x17\x88:EYy\x0e\x02\x0b3Y$\x86\x94"t&6\x19c\xb5\xe5\x81XC\xf5\xa1(\xa4\xf2\xa2x\xae\xe8R\x91\x08-C\xc7"\xd4\x94Q\xefO`\xe3/\xc6W\x0f\x91\xa6\xd7\xfd\x1c\xd3"\xa0\xa2\xb8\xf7\xcf\xb9?\xfe.\x94\xa44\x11\xd3>\xf8\x9d\x04\x1boGA\x16\x9f\x976\xfe\xd0NA\xde\xfd\xaa\xa2\xa2PE@w\x83\x0f_(\xd2\x03\xd5g/{\xc6\x13tMs>\xe98\x9a\xb7\x15\x9d\xaf\xfe\xf2z\xd6\xe7bT\xd1\x

> **You**: Hello.
> 
> **Gemini**:
> Hello there! How can I assist you today?